In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from environment import *

with open("setting.yaml") as yaml_file:

    setting = yaml.load(yaml_file)

path = make_path_dict(setting)

In [ ]:
w = pd.read_table(path["w.tsv"], index_col=0)

h = pd.read_table(path["h.tsv"], index_col=0)

In [ ]:
if setting["element_entropy_quantile"] < 1:

    feature_entropy = w.apply(ccal.compute_entropy, axis=1)

    w = w.loc[
        feature_entropy.index[
            feature_entropy
            < feature_entropy.quantile(q=setting["element_entropy_quantile"])
        ]
    ]

    print(w.shape)

    sample_entropy = h.apply(ccal.compute_entropy)

    h = h[
        sample_entropy.index[
            sample_entropy
            < sample_entropy.quantile(q=setting["element_entropy_quantile"])
        ]
    ]

    print(h.shape)

In [ ]:
gps_map = ccal.GPSMap(
    w=w.T,
    h=h,
    w_n_pull=setting["gps_map_w_n_pull"],
    w_pull_power=setting["gps_map_w_pull_power"],
    h_n_pull=setting["gps_map_h_n_pull"],
    h_pull_power=setting["gps_map_h_pull_power"],
    plot=False,
)

In [ ]:
for element_type in ("feature", "sample"):

    if element_type == "feature":

        w_or_h = "w"

    elif element_type == "sample":

        w_or_h = "h"

    feature_x_sample_alias = setting["feature_x_sample_alias"]

    element_alias = setting["{}_alias".format(element_type)]

    title = "{}<br>{} GPS Map".format(feature_x_sample_alias, element_alias)

    element_marker_size = setting["gps_map_{}_element_marker_size".format(w_or_h)]

    gps_map_directory_path = path["{}|gps_map/".format(w_or_h)]

    gps_map.plot_gps_map(
        w_or_h,
        element_marker_size=element_marker_size,
        title=title,
        html_file_path="{}/gps_map.html".format(gps_map_directory_path),
    )

    hcc__k_x_column_file_path = path["{}|hcc__k_x_column.tsv".format(w_or_h)]

    if os.path.isfile(hcc__k_x_column_file_path):

        hcc__k_x_column = pd.read_table(hcc__k_x_column_file_path, index_col=0)

        if element_type == "feature":

            elements = gps_map.w_elements

        elif element_type == "sample":

            elements = gps_map.h_elements

        hcc_k = setting["{}_hcc_k".format(w_or_h)]

        bandwidth_factor = setting["gps_map_{}_bandwidth_factor".format(w_or_h)]

        gps_map.set_element_labels(
            w_or_h,
            hcc__k_x_column.loc["K{}".format(hcc_k)],
            bandwidth_factor=bandwidth_factor,
        )

        if setting["plotly_directory_path"] is None:

            plotly_file_path = None

        else:

            plotly_file_path = "{}/{} GPS Map.html".format(
                path["plotly/"], element_type.title()
            )

        hcc_gps_map_directory_path = path["{}|hcc|gps_map/".format(w_or_h)]

        gps_map.plot_gps_map(
            w_or_h,
            element_marker_size=element_marker_size,
            title=title,
            html_file_path="{}/gps_map.html".format(hcc_gps_map_directory_path),
            plotly_file_path=plotly_file_path,
        )

In [ ]:
ccal.dump_gps_map(gps_map, path["gps_map.pickle.gz"])